# Bitcoin Price Prediction

### Introdução<br>

#### O objetivo desse projeto e prever futuros valores para o Bitcoin seguindo principalmente os valores das seguintes variaveis de instantes anteriores. As quais foram escolhidas por possuirem uma alta correlacao com o "market_price_2", que é a nossa variavel de interesse<br>

-cost_per_transaction_percent<br>
-difficulty<br>
-hash_rate<br>
-market_cap<br>
-estimated_transaction_volume_usd<br>


In [ ]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dfs = pd.read_excel('DataFrame_bitcoin.xlsx')

In [ ]:
dfs.head()

### Análise exploratória

In [ ]:
tabela_corr = dfs.corr()
tabela_corr

In [ ]:
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').values,
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [ ]:
correlation_heatmap(dfs, 'Correlação das variávies')

### Como podemos ver as variáveis que apresentam a correlação mais forte com o market price (preço) são:  
-Custo Por Transação Em Porcentagem  (miners revenue as percentage of the transaction volume.)<br>
-Dificuldade  
-Hash_rate<br>
-Market_Cap<br>
-Volumne Estimado Da Transacao<br>
#### Agora vamos analisar mais a fundo essas relações

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["cost_per_transaction_percent"])
plt.title('Preço x Custo percentual de transação')
plt.xlabel('Preço')
plt.ylabel('Custo percentual de transação')

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["difficulty"])
plt.title('Preço x Dificulade')
plt.xlabel('Preço')
plt.ylabel('Dificuladade')

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["hash_rate"])
plt.title('Preço x Hash Rate')
plt.xlabel('Preço')
plt.ylabel('Hash Rate')

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["market_cap"])
plt.title('Preço x Market_Cap')
plt.xlabel('Preço')
plt.ylabel('Market_Cap')

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["estimated_transaction_volume_usd"])
plt.title('Preço x Volumne Estimado Da Transacao')
plt.xlabel('Preço')
plt.ylabel('Volumne Estimado Da Transacao')

In [ ]:
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

Y = dfs['market_price_2']
X = dfs[[
 'avg_block_size',
 'cost_per_transaction_percent',
 'cost_per_transaction',
 'difficulty',
 'estimated_transaction_volume_usd',
 'hash_rate',
 'market_cap',
 'miners_revenue',
 'n_transactions_excluding_popular',
 'n_transactions',
 'output_volume',
 'total_bitcoins',
 'transaction_fees'
]]
#values come from day before
print(X.shape)

In [ ]:
results.summary()

In [ ]:
n = len(Y)
lista = []
dias = []
for i in range(200):
    Y_offset = Y.values[i:]
    X_offset = X.values[:(n-i),:]
    results = regress(X_offset,Y_offset)
    lista.append(results)
    dias.append(i)
    print(i, results.rsquared)

Decidimos fazer a analise usando 10 períodos anteriores, uma vez que analisando até 200 períodos esse foi com maior precisão, ainda sim usando dados anteriores

In [ ]:
for i in range(200):
    if i == 10:
        Y_offset1 = Y.values[i:]
        X_offset1 = X.values[:(n-i),:]
        results1 = regress(X_offset1,Y_offset1)
results1.summary()

Como podemos analisar no resultado da regressão tivemos algumas variáveis com um  valor p muito alto. Agora vamos retirar essas variáveis para ver se conseguimos aumentar a precisão

In [ ]:
X1 = dfs[[
 'avg_block_size',
 'cost_per_transaction',
 'difficulty',
 'estimated_transaction_volume_usd',
 'hash_rate',
 'market_cap',
 'miners_revenue',
 'n_transactions',
 'transaction_fees'
]]

In [ ]:
for i in range(200):
    if i == 10:
        Y_offset2 = Y.values[i:]
        X_offset2 = X1.values[:(n-i),:]
        results2 = regress(X_offset2,Y_offset2)
results2.summary()

Ao tirar as variáveis é possível perceber que não houve melhora na performace do código

In [ ]:
#Decision Tree Regressor
Arvore = DecisionTreeRegressor(random_state = 1)
Arvore.fit(X, Y)
plt.scatter(Y, Arvore.predict(X), color = 'green')
plt.show()